# DQN

*Deep Q Network*

In [1]:
import gym
import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML
from tqdm import tqdm
import random


---

## CartPole

今回扱う問題．ポールが倒れないようなカート操作ができるように学習させる．

- [Cart Pole - Gymnasium Documentation](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![](https://gymnasium.farama.org/_images/cart_pole.gif)

<br>

以下の4つの状態を持つ

| 状態 | 範囲 |
| --- | --- |
|カートの位置 | -4.8 ~ 4.8 |
| カートの速度 | -Inf ~ Inf |
| ポールの角度 | -24° ~ 24° |
| ポールの角速度 | -Inf ~ Inf |

また行動はカートを右に動かすか左に動かすかの2通り．

In [2]:
ENV = 'CartPole-v1'
env = gym.make(ENV, render_mode='rgb_array')
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n

print('n_actions:', n_actions)
print('n_states:', n_states)

n_actions: 2
n_states: 4



---

## DQN

### Q関数

Q関数となるニューラルネットワークを定義する．  
- 入力：状態
- 出力：全ての行動の価値

In [3]:
class QNet(nn.Module):
    def __init__(self, n_states, n_actions):
        super(QNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_states, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, n_actions)
        )
        
    def forward(self, x):
        y = self.net(x)
        return y

### エージェント

Q関数を所持し，それを元に行動を決定できるエージェントをクラスとして実装する．

In [4]:
class Agent:
    def __init__(self):
        self.q = QNet(n_states, n_actions)
        self.optim = optim.Adam(self.q.parameters(), lr=0.0001)
        self.loss_fn = nn.SmoothL1Loss()

    def get_action(self, s, epsilon=0):
        s = torch.tensor(s, dtype=torch.float32)
        if torch.rand(1).item() < epsilon:
            a = torch.randint(n_actions, (1,)).item()
        else:
            a = self.q(s).argmax().item()
        return a

    def update(self, s, a, r, next_s, gamma=0.99):
        """Q関数を更新する"""
        s = torch.tensor(s, dtype=torch.float32)
        next_s = torch.tensor(next_s, dtype=torch.float32)
        q = self.q(s)[a]
        target = r + gamma * self.q(next_s).max(-1).values.detach() # 正解
        loss = self.loss_fn(q, target) # 損失
        self.optim.zero_grad()
        loss.backward() # 逆伝播
        self.optim.step() # パラメータ更新

### 報酬

報酬は，ポールの角度の絶対値にマイナスをかけたものとする．  
ポールの角度が0°に近いほど，報酬は大きくなる．

In [5]:
def reward_func(s):
    cart_p, cart_v, pole_a, pole_v = s
    r = -abs(pole_a)
    return r

### 描画

ゲーム画面を描画する関数も実装

In [6]:
def run(agent, env, lim=500, interval=50):
    frames = []
    s, _ = env.reset()
    done = False
    for _ in range(lim):
        a = agent.get_action(s)
        s, _, done, _, _ = env.step(a)
        frames.append(env.render())
        if done:
            break

    fig = plt.figure()
    plt.axis('off')
    im = plt.imshow(frames[0])

    def update(i):
        im.set_array(frames[i])
        return im,

    ani = animation.FuncAnimation(
        fig, update, frames=len(frames), interval=interval)
    plt.close()
    display(HTML(ani.to_jshtml()))

### 学習

学習を行う関数の実装．  
行動決定→行動→状態遷移→報酬決定→Q関数更新 を繰り返す

In [7]:
def train(env, agent, n_episodes, epsilon=0.2, gamma=0.99, lim=500):
    for _ in tqdm(range(n_episodes)):
        s, _ = env.reset()
        done = False
        for _ in range(lim):
            a = agent.get_action(s, epsilon)
            next_s, _, done, _, _ = env.step(a)
            r = reward_func(next_s) if not done else -5
            agent.update(s, a, r, next_s, gamma)
            if done:
                break
            s = next_s

実際に学習させてみる．まずエージェント（Q関数）を初期化

In [8]:
agent = Agent()

初期状態での性能はこんな感じ

In [9]:
run(agent, env)

/Users/komiya/Private/study/reinforcement-learning/.venv/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ここから学習させる．

In [10]:
train(env, agent, 1000, epsilon=0.2)

100%|██████████| 1000/1000 [00:15<00:00, 62.86it/s]


学習結果

In [11]:
run(agent, env)